<a href="https://colab.research.google.com/github/Swayamprakashpatel/DE_Interact.github.io/blob/main/Gen_AI_Drug_Discovery_SKP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Generative AI Model for Drug Discovery**


**Dr. Swayamprakash Patel**
Associate Professor,
RPCP | CHARUSAT

Email: Swayamprakash.patel@gmail.com


In [ ]:
# @title Model Download
# Suppress all output from pip installation
!pip install gdown > /dev/null 2>&1
import gdown

# CVAE Model download
file_id = '17xSw68Rbfod9qxIZFAxrwgvbCyOFkll-'  # Replace with your actual Model file ID - CVAE Model
destination = '/content/Gen_AI_Model.pt'  # Desired destination path

# quiet=True suppresses the process printout
gdown.download(f'https://drive.google.com/uc?id={file_id}', destination, quiet=True)

# unique smile list download
file_id2 = '18tmu6boTvDRcPQVURBUoBZCH_EqXwu-0'  # Replace with your actual file ID
destination2 = '/content/unique_smiles_list.csv'  # Desired destination path

# quiet=True suppresses the process printout
gdown.download(f'https://drive.google.com/uc?id={file_id2}', destination2, quiet=True)

# Print the final message after all downloads are complete
print("SKP Model Downloaded")

In [ ]:
# @title Genrative_AI_Drug_Design
#from google.colab import drive

# --- Context Manager for Output Suppression ---
# This is a standard Python technique to temporarily silence printing.
class HiddenPrint:
    """Context manager to suppress stdout (print statements)."""
    def __enter__(self):
        self._original_stdout = sys.stdout
        # Use os.devnull to redirect stdout to null
        sys.stdout = open(os.devnull, 'w')
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

# --- SUPPRESS ALL SETUP OUTPUT (Installation and Loading Status) ---
# Wrapping the entire setup in a HiddenPrint block and suppressing !pip output.

#-------------------------------------------#
#...........INSTALL LIBRARIESS..............#
# Redirect stdout/stderr of pip install to /dev/null for maximum silence
!pip install torch > /dev/null 2>&1
!pip install transformers > /dev/null 2>&1
!pip install numpy > /dev/null 2>&1
!pip install rdkit > /dev/null 2>&1
!pip install Pillow > /dev/null 2>&1
!pip install transformers pillow > /dev/null 2>&1



import os
import sys
import json
import warnings
import numpy as np
import torch
import torch.nn as nn
from transformers import AutoTokenizer, EsmModel, AutoModel
from rdkit import Chem
from rdkit.Chem import Draw
from PIL import Image
from IPython.display import display # <-- CRITICAL FOR COLAB DISPLAY


# Suppress warnings and parallel tokenizers for cleaner output
warnings.filterwarnings("ignore")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# ==============================================================================
# 1. CONFIGURATION AND INITIALIZATION (SUPPRESSED)
# ==============================================================================

with HiddenPrint():
    # --- File Paths and Device ---
    # !!! IMPORTANT: UPDATE THIS PATH TO YOUR ACTUAL BEST MODEL FILE !!!
    BEST_MODEL_PATH = '/content/Gen_AI_Model.pt' # from Direct Content

    # ******************************************************************************
    EMBEDDING_SAVE_PATH = 'protein_embeddings.npy'
    SEQ_INDEX_PATH = 'protein_seq_index.json'
    OUTPUT_DIR = 'generated_structures'

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # print(f"Using device: {device}") # Suppressed

    # --- Model Dims (MUST match your training setup) ---
    HIDDEN_DIM = 256
    LATENT_DIM = 1024
    NUM_LAYERS = 3

    # --- ESM-2 Configuration ---
    ESM_MODEL_NAME = 'esm2_t12_35M_UR50D' # 35 Million Parameter
    #ESM_MODEL_NAME = 'esm2_t30_150M_UR50D' #150 Million Parameter
    #ESM_MODEL_NAME = 'esm2_t33_650M_UR50D' # 650 Million Parameter
    MAX_PROTEIN_SEQUENCE_LEN = 1000

    # --- ChemBERTa Configuration ---
    CHEMBERTA_MODEL_NAME = 'DeepChem/ChemBERTa-77M-MTR'

    # ----------------------
    # Load Tokenizers and Base Models
    # ----------------------
    try:
        # print(f"Loading ESM-2 model: {ESM_MODEL_NAME}...") # Suppressed
        esm_tokenizer = AutoTokenizer.from_pretrained(f"facebook/{ESM_MODEL_NAME}")
        esm_model = EsmModel.from_pretrained(f"facebook/{ESM_MODEL_NAME}").to(device).eval()
        ESM_EMBEDDING_DIM = esm_model.config.hidden_size
        ESM_AVAILABLE = True
    except Exception as e:
        ESM_AVAILABLE = False
        ESM_EMBEDDING_DIM = None
        # print(f"WARNING: ESM-2 not available ({e}). New sequences cannot be processed.") # Suppressed

    # print(f"Loading ChemBERTa tokenizer and model: {CHEMBERTA_MODEL_NAME}...") # Suppressed
    smiles_tokenizer = AutoTokenizer.from_pretrained(CHEMBERTA_MODEL_NAME)
    smiles_model = AutoModel.from_pretrained(CHEMBERTA_MODEL_NAME)
    SMILES_EMBEDDING_DIM = smiles_model.config.hidden_size
    SMILES_VOCAB_SIZE = smiles_tokenizer.vocab_size

    PAD_ID = smiles_tokenizer.pad_token_id
    BOS_ID = smiles_tokenizer.bos_token_id or smiles_tokenizer.cls_token_id
    EOS_ID = smiles_tokenizer.eos_token_id or smiles_tokenizer.sep_token_id

    # Infer the protein embedding dimension
    if ESM_EMBEDDING_DIM is None:
        try:
            all_protein_embeddings = np.load(EMBEDDING_SAVE_PATH, allow_pickle=True)
            protein_emb_dim = int(np.array(all_protein_embeddings[0]).shape[-1])
            # print(f"Inferred protein embedding dim from .npy: {protein_emb_dim}") # Suppressed
        except Exception:
            raise RuntimeError("Could not load ESM or infer protein dimension from .npy.")
    else:
        protein_emb_dim = ESM_EMBEDDING_DIM


# ==============================================================================
# 2. UTILITY AND MODEL CLASSES (No print statements in definitions)
# ==============================================================================

# --- Utility Functions ---
def safe_to_numpy(x):
    if isinstance(x, np.ndarray): return x
    return np.array(x)

def load_seq_index():
    if not os.path.exists(SEQ_INDEX_PATH):
        return {}
    with open(SEQ_INDEX_PATH, 'r', encoding='utf-8') as f:
        return json.load(f)

@torch.no_grad()
def get_esm_embedding_pooled(seq: str) -> np.ndarray:
    """Computes pooled ESM embedding for a new sequence (Novel Sequence Fallback)."""
    if not ESM_AVAILABLE:
        raise RuntimeError("ESM not available to compute embeddings.")
    seq_spaced = " ".join(seq)
    encoded = esm_tokenizer(seq_spaced, return_tensors='pt',
                             max_length=MAX_PROTEIN_SEQUENCE_LEN,
                             padding=True, truncation=True)
    encoded = {k: v.to(device) for k, v in encoded.items()}
    out = esm_model(**encoded)
    emb = out.last_hidden_state.mean(dim=1).squeeze().detach().cpu().numpy()
    return emb

def get_protein_embedding_for_seq(seq, seq_to_idx, all_protein_embeddings):
    """Retrieves embedding (DB Look-up for training data or ESM Fallback for novel sequences)."""
    if seq in seq_to_idx:
        print("Status: Sequence found in training index. Using precomputed embedding.")
        emb = safe_to_numpy(all_protein_embeddings[seq_to_idx[seq]])
        return torch.tensor(emb, dtype=torch.float32, device=device).unsqueeze(0)

    if ESM_AVAILABLE:
        print("Status: Sequence is new. Computing ESM-2 embedding on-the-fly...")
        emb = get_esm_embedding_pooled(seq)
        return torch.tensor(emb, dtype=torch.float32, device=device).unsqueeze(0)

    raise RuntimeError(
        "Cannot embed unseen protein sequence: No precomputed embedding and ESM-2 is unavailable. "
    )

def draw_smiles_to_file(smi_list, output_dir=OUTPUT_DIR):
    """Draws molecular structures for a list of valid SMILES, saves them, and displays them (using IPython display)."""
    if not smi_list:
        return

    os.makedirs(output_dir, exist_ok=True)

    mols = [Chem.MolFromSmiles(smi) for smi in smi_list]
    mols = [mol for mol in mols if mol is not None]

    if not mols:
        print("Could not convert any valid SMILES to RDKit molecules for drawing.")
        return

    print(f"\nDrawing, saving, and displaying {len(mols)} valid molecular structures...")

    # Draw and save/show each structure individually with an index
    for i, mol in enumerate(mols):
        try:
            img = Draw.MolToImage(mol, size=(300, 300))

            filename = os.path.join(output_dir, f'molecule_{i + 1}.png')
            img.save(filename) # Saves the file

            # --- COLAB/NOTEBOOK DISPLAY: Use IPython.display ---
            print(f"Displaying structure {i+1}...")
            display(img)
            # ----------------------------------------------------

        except Exception as e:
            print(f"Error processing molecule {i+1}: {e}")
            print("Note: Image display failed. Check environment configuration.")

# --- Model Architecture ---
class FiLM(nn.Module):
    def __init__(self, cond_dim, hidden_dim):
        super().__init__()
        self.gamma = nn.Linear(cond_dim, hidden_dim)
        self.beta  = nn.Linear(cond_dim, hidden_dim)
    def forward(self, x, cond):
        g = self.gamma(cond).unsqueeze(1)
        b = self.beta(cond).unsqueeze(1)
        return x * (1 + g) + b

class ConditionalVAE(nn.Module):
    def __init__(self, smiles_embedding_layer, vocab_size, smiles_embedding_dim,
                 protein_embedding_dim, hidden_dim, latent_dim, num_layers):
        super().__init__()
        self.latent_dim = latent_dim
        self.num_layers = num_layers

        # Architecture setup (matching your training code)
        self.smiles_embedding_layer = smiles_embedding_layer
        self.encoder_rnn = nn.GRU(smiles_embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.encoder_protein_mlp = nn.Sequential(nn.Linear(protein_embedding_dim, hidden_dim), nn.ReLU(), nn.Linear(hidden_dim, hidden_dim))
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)
        self.decoder_init = nn.Sequential(nn.Linear(latent_dim + protein_embedding_dim, hidden_dim), nn.ReLU())
        self.decoder_rnn = nn.GRU(smiles_embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.film = FiLM(protein_embedding_dim, hidden_dim)
        self.out_proj = nn.Linear(hidden_dim, smiles_embedding_dim, bias=False)
        self.fc_out = nn.Linear(smiles_embedding_dim, vocab_size, bias=False)
        self.fc_out.weight = self.smiles_embedding_layer.weight

    @torch.no_grad()
    def generate(self, protein_embedding, max_len, top_k, top_p, temperature):
        B = protein_embedding.size(0)
        z = torch.randn(B, self.latent_dim, device=protein_embedding.device)
        zc = torch.cat([z, protein_embedding], dim=1)
        h = self.decoder_init(zc).unsqueeze(0).repeat(self.decoder_rnn.num_layers, 1, 1)

        cur = torch.full((B, 1), BOS_ID, dtype=torch.long, device=protein_embedding.device)
        seq = [cur]

        for _ in range(max_len):
            emb = self.smiles_embedding_layer(cur[:, -1:])
            out, h = self.decoder_rnn(emb, h)
            out = self.film(out, protein_embedding)
            out = self.out_proj(out)
            logits = self.fc_out(out[:, -1, :]) / max(1e-8, temperature)

            probs = torch.softmax(logits, dim=-1)
            next_tok = self._sample_top_k_top_p(probs, top_k, top_p).unsqueeze(1)
            seq.append(next_tok)
            cur = torch.cat([cur, next_tok], dim=1)
            if torch.all(next_tok.squeeze(1) == EOS_ID):
                break
        return torch.cat(seq, dim=1)

    def _sample_top_k_top_p(self, probs, top_k, top_p):
        B, V = probs.size()
        if top_k is not None and top_k > 0:
            topk_vals, topk_idx = torch.topk(probs, k=min(top_k, V), dim=-1)
            mask = torch.zeros_like(probs).scatter_(1, topk_idx, 1.0)
            probs = probs * mask
        if top_p is not None and 0.0 < top_p < 1.0:
            sorted_probs, sorted_idx = torch.sort(probs, descending=True, dim=-1)
            cum = torch.cumsum(sorted_probs, dim=-1)
            cutoff = cum > top_p
            cutoff[..., 0] = False
            sorted_probs[cutoff] = 0.0
            probs = torch.zeros_like(probs).scatter(1, sorted_idx, sorted_probs)

        probs = probs / probs.sum(dim=-1, keepdim=True).clamp_min(1e-8)
        return torch.multinomial(probs, num_samples=1).squeeze(1)


# ==============================================================================
# 3. MAIN PREDICTION FUNCTION
# ==============================================================================

@torch.no_grad()
def generate_smiles_for_sequence(raw_sequence: str, num_samples: int, max_len: int, temperature: float, top_k: int = 50, top_p: float = 0.99) -> list[str]:
    """
    Generates and validates SMILES molecules for a given protein sequence.
    Returns a list of valid SMILES strings.
    """
    if not os.path.exists(BEST_MODEL_PATH):
        raise FileNotFoundError(f"Best model not found at {BEST_MODEL_PATH}. Train first!")

    # 1. Load data artifacts
    seq_to_idx = load_seq_index()
    try:
        all_protein_embeddings = np.load(EMBEDDING_SAVE_PATH, allow_pickle=True)
    except FileNotFoundError:
        if not ESM_AVAILABLE:
              raise FileNotFoundError(f"Embedding file {EMBEDDING_SAVE_PATH} not found and ESM unavailable.")
        all_protein_embeddings = []

    # 2. Rebuild and Load Model
    pre_emb = smiles_model.get_input_embeddings().weight.detach().clone()
    smiles_embedding_layer = nn.Embedding.from_pretrained(pre_emb, freeze=False)

    model = ConditionalVAE(
        smiles_embedding_layer=smiles_embedding_layer,
        vocab_size=SMILES_VOCAB_SIZE,
        smiles_embedding_dim=SMILES_EMBEDDING_DIM,
        protein_embedding_dim=protein_emb_dim,
        hidden_dim=HIDDEN_DIM,
        latent_dim=LATENT_DIM,
        num_layers=NUM_LAYERS
    ).to(device)
    model.load_state_dict(torch.load(BEST_MODEL_PATH, map_location=device))
    model.eval()

    # 3. Get embedding for the input sequence (Handles novel sequences via ESM-2)
    try:
        pe = get_protein_embedding_for_seq(raw_sequence, seq_to_idx, all_protein_embeddings)
    except RuntimeError as e:
        print(f"ERROR: {e}")
        return []

    # 4. Generate with controlled parameters
    valid_smiles_list = []

    print(f"  Samples: {num_samples} | Max Len: {max_len} tokens | Temp: {temperature} | Top-P: {top_p}")

    for i in range(num_samples):
        ids = model.generate(pe, max_len=max_len, top_k=top_k, top_p=top_p, temperature=temperature)
        smi = smiles_tokenizer.decode(ids[0].tolist(), skip_special_tokens=True)

        # Validation Check
        is_valid = Chem.MolFromSmiles(smi) is not None
        validity = "VALID" if is_valid else "INVALID"

        if is_valid:
            valid_smiles_list.append(smi)

        print(f"  Sample {i+1} of {num_samples}: [{validity}] {smi}")

    return valid_smiles_list

# ==============================================================================
# 4. MAIN EXECUTION BLOCK (User Interface)
# ==============================================================================

if __name__ == "__main__":

    # --- Recommended Default Controls ---
    DEFAULT_MAX_LEN = 90
    DEFAULT_MIN_LEN = 30 # New default for range mode
    DEFAULT_TEMP = 0.8
    DEFAULT_SAMPLES = 10

    # This is the first print statement visible to the user
    print("\n--- Conditional VAE Molecule Generation Interface ---")

    # In a Colab notebook, you will need to upload your model files first.
    raw_sequence = input("Enter the new protein sequence: \n> ")

    try:
        num_samples = int(input(f"Enter number of SMILES to generate per length (default: {DEFAULT_SAMPLES}): ") or DEFAULT_SAMPLES)
        temperature = float(input(f"Enter Temperature (e.g., 0.8 for high validity, 1.2 for high diversity, default: {DEFAULT_TEMP}): ") or DEFAULT_TEMP)
    except ValueError:
        print("Invalid number or temperature entered. Using defaults for samples/temp.")
        num_samples = DEFAULT_SAMPLES
        temperature = DEFAULT_TEMP

    # --- NEW LENGTH LOGIC ---
    print("\n--- SMILES LENGTH MODE ---")
    length_mode = input("Select generation mode: [F]ixed Length or [V]ariable Length Range? (Default: F): ")
    length_mode = length_mode.strip().upper()

    generation_lengths = []

    if length_mode == 'V':
        try:
            min_len = int(input(f"Enter Minimum SMILES Length in tokens (default: {DEFAULT_MIN_LEN}): ") or DEFAULT_MIN_LEN)
            max_len = int(input(f"Enter Maximum SMILES Length in tokens (default: {DEFAULT_MAX_LEN}): ") or DEFAULT_MAX_LEN)

            if min_len > max_len:
                print("Warning: Minimum length cannot be greater than maximum. Using fixed default length.")
                generation_lengths = [DEFAULT_MAX_LEN]
            else:
                # Generate a list of lengths from min_len to max_len, stepping by 5
                generation_lengths = list(range(min_len, max_len + 1, 5))
                print(f"Set to generate for lengths: {min_len} to {max_len} (stepping by 5).")
                print(f"Generation lengths: {generation_lengths}")
        except ValueError:
            print("Invalid length entered. Using fixed default length.")
            generation_lengths = [DEFAULT_MAX_LEN]

    else: # Default to Fixed Length (or invalid input)
        try:
            max_len = int(input(f"Enter Max SMILES Length in tokens (default: {DEFAULT_MAX_LEN}): ") or DEFAULT_MAX_LEN)
            generation_lengths = [max_len]
            print(f"Set to generate for fixed length: {max_len}.")
        except ValueError:
            print("Invalid length entered. Using fixed default length.")
            generation_lengths = [DEFAULT_MAX_LEN]

    # --- Execution Loop ---
    print("\nStarting generation process...")

    all_valid_smiles = []
    total_generated = 0

    for current_max_len in generation_lengths:
        # NOTE: The generate function uses max_len as the termination condition.
        # We call it sequentially for each length in the desired range.
        print(f"\n=======================================================")
        print(f"| RUNNING: Max Length = {current_max_len} tokens |")
        print(f"=======================================================")

        # Run the generation for the current max_len
        valid_smiles = generate_smiles_for_sequence(
            raw_sequence.strip(),
            num_samples,
            current_max_len,
            temperature
        )

        all_valid_smiles.extend(valid_smiles)
        total_generated += num_samples

    # --- Final Output and Drawing ---
    total_valid = len(all_valid_smiles)

    if all_valid_smiles:
        print(f"\n==========================================")
        print(f"--- FINAL RESULTS (Combined from {len(generation_lengths)} length range(s)) ---")
        print(f"Total Samples Generated: {total_generated}")
        print(f"Total Valid Samples: {total_valid}")
        print(f"Validity Percentage (across all): {total_valid / total_generated * 100:.2f}%")
        print("==========================================")

        # 1) List of Final, Valid SMILES
        print("\nFinal List of Valid SMILES:")
        for i, smi in enumerate(all_valid_smiles):
            print(f"  {i+1}. {smi}")

        # 2) Draw, SAVE, and SHOW Structures
        draw_smiles_to_file(all_valid_smiles)
        print(f"\n✅ Structures saved to the '{OUTPUT_DIR}' directory and displayed!")
    else:
        print("\nGeneration yielded no valid SMILES.")

In [ ]:
# @title Comprehensive Pre-Docking Analysis

# --- INSTALL/IMPORT ALL NECESSARY LIBRARIES ---
# Suppressing installation output
!pip install pandas > /dev/null 2>&1
!pip install rdkit > /dev/null 2>&1
!pip install matplotlib seaborn > /dev/null 2>&1

import os
import sys
import pandas as pd
import numpy as np
import math
import time
from random import uniform
from IPython.display import display, Markdown, Image
from PIL import Image as PILImage
import matplotlib.pyplot as plt
import seaborn as sns

from rdkit import Chem
# QED is accessed via Descriptors.qed(mol)
from rdkit.Chem import Descriptors, AllChem, Draw
from rdkit.Chem.FilterCatalog import FilterCatalogParams, FilterCatalog
from rdkit.DataStructs import BulkTanimotoSimilarity, TanimotoSimilarity
from rdkit.Chem.rdFMCS import FindMCS
from rdkit.Chem.rdMolDescriptors import CalcNumSpiroAtoms, CalcNumBridgeheadAtoms, CalcNumAmideBonds # Used for SA Score

# --- Suppress RDKit specific warnings ---
from rdkit import RDLogger
rdLogger = RDLogger.logger()
rdLogger.setLevel(RDLogger.CRITICAL)

# ==============================================================================
# --- SA SCORE HELPER FUNCTION (Ensures self-contained script) ---
# NOTE: This implementation is adapted from the RDKit contributor script (sascorer.py)
# to make the notebook self-contained.
# ==============================================================================
def calculate_sa_score(mol):
    """Calculates the Synthetic Accessibility Score (SA Score) for a molecule."""
    # These imports are needed specifically for the SA Score calculation logic
    from rdkit.Chem.Scaffolds import MurckoScaffold
    from rdkit.Chem.rdMolDescriptors import CalcNumSpiroAtoms, CalcNumBridgeheadAtoms

    # RDKit's SA Score function is complex. We'll use a functional proxy that leverages
    # fragment counts and complexity features, which is the spirit of SA score.

    # Component 1: Fragment Score (Usually requires the full fragment library, skipped here for simplicity)
    # Component 2: Complexity Score (Number of rings, heteroatoms, spiro/bridgehead atoms, etc.)

    complexity = 0
    complexity += mol.GetNumAtoms() / 100.0 # Atoms count penalty
    complexity += Descriptors.HeavyAtomCount(mol) / 100.0
    complexity += Descriptors.RingCount(mol) * 0.5
    complexity += CalcNumSpiroAtoms(mol) * 1.0
    complexity += CalcNumBridgeheadAtoms(mol) * 1.5

    # Component 3: Penalties (e.g., number of stereocenters, large rings)
    chiral_centers = len(Chem.FindMolChiralCenters(mol, includeUnassigned=True))
    complexity += chiral_centers * 1.0

    # SA Score is often presented on a scale of 1 (easy) to 10 (hard)
    # This formula is a heuristic proxy (lower is better, easier to synthesize)
    sa_score = 10 - (10 * np.exp(-complexity / 5.0)) # Maps complexity to the 1-10 range

    # Ensure score is within bounds [1, 10]
    return max(1.0, min(10.0, sa_score))


# ==============================================================================

# 0. INITIAL DATA CHECK AND SETUP (FORCES USE OF GENERATED SMILES)
smiles_list_raw = globals().get('all_valid_smiles') or globals().get('valid_smiles')

if not smiles_list_raw:
    print("FATAL ERROR: The required generated SMILES list (`all_valid_smiles` or `valid_smiles`) is NOT available in the global environment.")
    print("Please ensure the CVAE generation cell has run successfully and populated the global variable.")
    sys.exit()

print(f"✅ Starting pipeline analysis on a combined total of {len(smiles_list_raw)} valid SMILES molecules (from CVAE output).")
current_smiles = smiles_list_raw
valid_mols = [Chem.MolFromSmiles(s) for s in current_smiles]
valid_mols_final = [m for m in valid_mols if m is not None]
current_smiles_final = [s for s, m in zip(current_smiles, valid_mols) if m is not None]

if not valid_mols_final:
    print("FATAL ERROR: Although SMILES were found, no valid RDKit molecules could be generated.")
    sys.exit()

# Set up the DataFrame base for consistency
df_base = pd.DataFrame({'SMILES': current_smiles_final})

# ==============================================================================

# 1. RO5, QED, and PAINS ANALYSIS (Updates df_filtered)
print("\n--- STAGE 1: Drug-Likeness (Ro5, QED & PAINS) Analysis ---")

LIPINSKI_CRITERIA = {'MW': 500, 'LogP': 5.0, 'HBD': 5, 'HBA': 10}
params = FilterCatalogParams()
params.AddCatalog(FilterCatalogParams.FilterCatalogs.PAINS)
pains_catalog = FilterCatalog(params)

def check_lipinski_rule(mol):
    # Calculate TPSA, Fsp3, Rotatable bonds for additional analysis
    mw, logp, hbd, hba = Descriptors.MolWt(mol), Descriptors.MolLogP(mol), Descriptors.NumHDonors(mol), Descriptors.NumHAcceptors(mol)
    tpsa = Descriptors.TPSA(mol)
    fsp3 = Descriptors.FractionCSP3(mol)
    rot_bonds = Descriptors.NumRotatableBonds(mol)
    qed_score = Descriptors.qed(mol) # --- NEW: QED Calculation ---

    violations = sum([
        mw > LIPINSKI_CRITERIA['MW'], logp > LIPINSKI_CRITERIA['LogP'],
        hbd > LIPINSKI_CRITERIA['HBD'], hba > LIPINSKI_CRITERIA['HBA']
    ])
    ro5_pass = violations <= 1
    # Return QED score as well
    return mw, logp, hbd, hba, tpsa, fsp3, rot_bonds, violations, ro5_pass, qed_score

def check_pains(mol):
    matches = pains_catalog.GetMatches(mol)
    is_pains = len(matches) > 0
    pains_descriptions = [match.GetDescription() for match in matches]
    return is_pains, "; ".join(pains_descriptions)

ro5_results = []
for smi in df_base['SMILES']:
    mol = Chem.MolFromSmiles(smi)
    if mol is None: continue

    # Unpack QED score
    mw, logp, hbd, hba, tpsa, fsp3, rot_bonds, ro5_viol, ro5_pass, qed_score = check_lipinski_rule(mol)
    is_pains, pains_desc = check_pains(mol)

    ro5_results.append({
        'SMILES': smi,
        'MW': f"{mw:.2f}",
        'LogP': f"{logp:.2f}",
        'HBD': hbd,
        'HBA': hba,
        'TPSA': f"{tpsa:.2f}",
        'Fsp3': f"{fsp3:.3f}",
        'Rot. Bonds': rot_bonds,
        'Ro5 Violations': ro5_viol,
        'Ro5 Pass': ro5_pass,
        'Is PAINS': is_pains,
        'PAINS Desc': pains_desc,
        'QED_Value': qed_score, # For sorting
        'QED Score': f"{qed_score:.3f}", # For display
        'Final Pass': ro5_pass and not is_pains
    })

df_filtered = pd.DataFrame(ro5_results)
globals()['df_filtered'] = df_filtered

print(f"   -> Ro5 Pass: {df_filtered['Ro5 Pass'].sum()} | No PAINS Alert: {len(df_filtered) - df_filtered['Is PAINS'].sum()}")
print(f"   -> Average QED Score: {df_filtered['QED_Value'].mean():.3f}")
print("   -> Drug-Likeness Analysis Complete.")

# ==============================================================================

# 2. DIVERSITY, NOVELTY, TOXICITY, AND SA SCORE ANALYSIS (Updates df_output and avg_internal_distance)
print("\n--- STAGE 2: Diversity, Novelty, Toxicity, and SA Score Analysis ---")

file_name = '/content/unique_smiles_list.csv' # <-- **CHANGE THIS TO YOUR TRAINING DATA CSV**
smiles_column_name = 'SMILES'
sample_size = 100000

training_smiles = []
try:
    df_training = pd.read_csv(file_name, usecols=[smiles_column_name]).dropna(subset=[smiles_column_name])
    effective_sample_size = min(sample_size, len(df_training))
    training_smiles = df_training.sample(n=effective_sample_size, random_state=42)[smiles_column_name].tolist()
    print(f"   -> Loaded {len(training_smiles)} training SMILES for Novelty calculation.")
except Exception:
    training_smiles = ['CC(=O)Oc1ccccc1C(=O)O'] * 100
    print(f"   -> WARNING: Training data file '{file_name}' not found. Using minimal mock SMILES for Novelty.")

def calculate_novelty(query_fps, training_smiles):
    if not training_smiles: return [0.0] * len(query_fps)
    training_mols = [Chem.MolFromSmiles(s) for s in training_smiles if Chem.MolFromSmiles(s) is not None]
    training_fps = [AllChem.GetMorganFingerprint(m, 2) for m in training_mols]
    max_sims = []
    for fp in query_fps:
        sims = BulkTanimotoSimilarity(fp, training_fps)
        max_sims.append(max(sims))
    return max_sims

def calculate_diversity(query_fps):
    n = len(query_fps)
    if n < 2: return 0.0
    total_distance = sum((1.0 - TanimotoSimilarity(query_fps[i], query_fps[j])) for i in range(n) for j in range(i + 1, n))
    count = n * (n - 1) / 2
    return total_distance / count

def check_tox_alerts(mol):
    alerts = {'[#6]C#N': 'Cyano Group', '[#7](=O)=O': 'Nitro Group'}
    is_alert, alert_descriptions = False, []
    for smarts, desc in alerts.items():
        if mol.HasSubstructMatch(Chem.MolFromSmarts(smarts)):
            is_alert = True
            alert_descriptions.append(desc)
    return is_alert, "; ".join(alert_descriptions)

fps = [AllChem.GetMorganFingerprint(m, 2) for m in valid_mols_final]
max_novelty_sims = calculate_novelty(fps, training_smiles)
avg_internal_distance = calculate_diversity(fps)
globals()['avg_internal_distance'] = avg_internal_distance

analysis_results = []
for i, mol in enumerate(valid_mols_final):
    is_tox_alert, tox_desc = check_tox_alerts(mol)
    novelty_sim = max_novelty_sims[i] if training_smiles else 1.0
    novelty_score_display = f"{novelty_sim:.3f}" if training_smiles else "0.000 (Skipped)"

    # --- SA Score Calculation ---
    sa_score = calculate_sa_score(mol)
    # -------------------------------

    initial_priority = 'MEDIUM'
    if is_tox_alert:
        initial_priority = 'REJECTED (Tox Alert)'

    analysis_results.append({
        'SMILES': current_smiles_final[i],
        'Max Novelty Sim': novelty_score_display,
        'Tox Alert': is_tox_alert,
        'Tox Alert Desc': tox_desc,
        'FINAL RDKIT PRIORITY': initial_priority,
        'Sort_Novelty_Value': novelty_sim,
        'SA Score': sa_score # --- NEW COLUMN ---
    })

df_output_novelty = pd.DataFrame(analysis_results)

# Merge with the full property table (df_filtered)
df_output = df_output_novelty.merge(
    df_filtered[['SMILES', 'MW', 'LogP', 'TPSA', 'Fsp3', 'Rot. Bonds', 'QED_Value']], # ADD QED_Value
    on='SMILES',
    how='left'
)
globals()['df_output'] = df_output

print(f"   -> Average Tanimoto Distance (Diversity): {avg_internal_distance:.3f}")
print("   -> Novelty, Toxicity, and SA Score Analysis Complete.")

# ==============================================================================

# 3. MAXIMAL COMMON SUBSTRUCTURE (MCS) ANALYSIS
print("\n--- STAGE 3: Maximal Common Substructure (MCS) Analysis ---")

def perform_mcs_analysis(mols: list):
    if len(mols) < 2:
        print(f"   -> MCS skipped: Found only {len(mols)} valid molecule(s).")
        return None, None

    mcs_result = FindMCS(mols, timeout=60, atomCompare=Chem.rdFMCS.AtomCompare.CompareElements)
    mcs_smarts = mcs_result.smartsString
    mcs_mol = Chem.MolFromSmarts(mcs_smarts)

    print(f"   -> MCS SMARTS: {mcs_smarts}")

    return mcs_mol, mcs_smarts

mcs_mol, mcs_smarts = perform_mcs_analysis(valid_mols_final)

# ==============================================================================

# 4. RDKIT-BASED PRIORITIZATION (Final Priority Assignment)
print("\n--- STAGE 4: Final RDKit-Based Prioritization ---")

df_combined = df_output.merge(
    df_filtered[['SMILES', 'Ro5 Pass', 'Is PAINS', 'PAINS Desc', 'QED Score']], # Add QED Score for display
    on='SMILES',
    how='inner'
)

# Handle missing values and convert to numeric
df_combined['TPSA'] = pd.to_numeric(df_combined['TPSA'], errors='coerce').fillna(999.0)
df_combined['MW'] = pd.to_numeric(df_combined['MW'], errors='coerce').fillna(999.0)
df_combined['LogP'] = pd.to_numeric(df_combined['LogP'], errors='coerce').fillna(999.0)
df_combined['Rot. Bonds'] = pd.to_numeric(df_combined['Rot. Bonds'], errors='coerce').fillna(999.0)
df_combined['Max Novelty Sim'] = pd.to_numeric(df_combined['Max Novelty Sim'], errors='coerce').fillna(1.0)
df_combined['SA Score'] = pd.to_numeric(df_combined['SA Score'], errors='coerce').fillna(10.0) # SA Score Max is 10 (hard)
df_combined['QED_Value'] = pd.to_numeric(df_combined['QED_Value'], errors='coerce').fillna(0.0) # --- NEW: QED Value ---

# Rejection Criteria
df_combined.loc[
    (df_combined['Ro5 Pass'] == False) | (df_combined['Is PAINS'] == True),
    'FINAL RDKIT PRIORITY'
] = 'REJECTED (Ro5/PAINS)'
df_combined.loc[
    df_combined['Tox Alert'] == True,
    'FINAL RDKIT PRIORITY'
] = 'REJECTED (Tox Alert)'

df_compliant = df_combined[
    ~df_combined['FINAL RDKIT PRIORITY'].str.startswith('REJECTED')
].copy()

# --- UPDATED SORTING CRITERIA: HIGHER QED, Lower TPSA, Lower SA Score, Lower Novelty Sim is better ---
df_compliant = df_compliant.sort_values(
    by=['QED_Value', 'TPSA', 'SA Score', 'Sort_Novelty_Value'],
    ascending=[False, True, True, True] # False for QED (Higher is Better), True for others (Lower is Better)
)

top_n_compliant = max(1, min(10, len(df_compliant) // 10))

# Assign final priorities
df_combined.loc[df_compliant.index, 'FINAL RDKIT PRIORITY'] = 'LOW'
df_combined.loc[df_compliant.index[:top_n_compliant], 'FINAL RDKIT PRIORITY'] = 'HIGH'
df_combined.loc[df_compliant.index[top_n_compliant:top_n_compliant*2], 'FINAL RDKIT PRIORITY'] = 'MEDIUM'

# Final sorting for display
priority_order = pd.CategoricalDtype(['HIGH', 'MEDIUM', 'LOW', 'REJECTED (Tox Alert)', 'REJECTED (Ro5/PAINS)'], ordered=True)
df_combined['FINAL RDKIT PRIORITY'] = df_combined['FINAL RDKIT PRIORITY'].astype(priority_order)

df_combined = df_combined.sort_values(
    by=['FINAL RDKIT PRIORITY', 'QED_Value', 'SA Score', 'TPSA', 'Sort_Novelty_Value'], # QED_Value added here
    ascending=[True, False, True, True, True] # Priority (HIGH first), then QED (Higher is Better), then SA (Easy first), then TPSA, then Novelty
)
globals()['df_combined'] = df_combined # Ensure df_combined is available globally

# ==============================================================================

# 5. PRINT FINAL REPORT
print("\n" + "=" * 80)
print("             🏆 FINAL DRUG DISCOVERY REPORT - SUMMARY MATRIX 🏆")
print("=" * 80)

summary_data = {
    'Metric Category': ['Generation', 'Drug-Likeness', 'Drug-Likeness', 'Physicochemical', 'Synthesizability', 'Synthesizability', 'Chemical Space', 'Toxicity/Priority', 'Final Leads'],
    'Metric Name': [
        'Total Unique Valid Molecules',
        'Average QED Score (Higher is Better)', # --- NEW ---
        'Best QED Score (Max)', # --- NEW ---
        'Average TPSA ($\AA^2$)',
        'Average SA Score (1=Easy, 10=Hard)',
        'Best SA Score (Easiest)',
        'Maximal Diversity (Tanimoto Distance)',
        'Molecules Marked HIGH RDKIT Priority',
        'Final High-Confidence Leads (Ro5 Pass & Top RDKit Priority)'
    ],
    'Value': [
        f'{len(smiles_list_raw)}',
        f'{df_combined["QED_Value"].mean():.3f}', # --- NEW ---
        f'{df_combined["QED_Value"].max():.3f}', # --- NEW ---
        f'{df_combined["TPSA"].mean():.2f}',
        f'{df_combined["SA Score"].mean():.2f}',
        f'{df_combined["SA Score"].min():.2f}',
        f'{avg_internal_distance:.3f}',
        f'{df_combined[df_combined["FINAL RDKIT PRIORITY"] == "HIGH"].shape[0]} / {len(df_combined)}',
        f'{df_combined[df_combined["FINAL RDKIT PRIORITY"] == "HIGH"].shape[0]} / {len(df_combined)}'
    ]
}
df_summary = pd.DataFrame(summary_data)
summary_markdown = df_summary.to_markdown(index=False)

print("\n\n-----------------------------------------------")
print("TABLE 1: PROJECT SUMMARY AND PERFORMANCE METRICS")
print("-----------------------------------------------\n")
display(Markdown(summary_markdown))

# Full candidates table (Table 2)
df_full_candidates = df_combined[[
    'FINAL RDKIT PRIORITY',
    'SMILES',
    'MW',
    'LogP',
    'QED Score', # --- NEW COLUMN ADDED HERE ---
    'TPSA',
    'SA Score',
    'Rot. Bonds',
    'Fsp3',
    'Ro5 Pass',
    'Is PAINS',
    'Tox Alert',
    'Max Novelty Sim',
]].drop(columns=['Sort_Novelty_Value', 'QED_Value'], errors='ignore')

full_candidates_markdown = df_full_candidates.to_markdown(index=False)

print("\n\n-----------------------------------------------")
print(f"TABLE 2: FULL LEAD CANDIDATE MATRIX (Total: {len(df_combined)} Molecules)")
print("(Sorted by RDKIT Priority (HIGH first), then QED Score (Higher is Better))")
print("-----------------------------------------------\n")
display(Markdown(full_candidates_markdown))

# Final 2D structure grid image for leads
if len(df_combined) > 0:
    # Convert SMILES to molecules
    grid_mols = [Chem.MolFromSmiles(smi) for smi in df_combined['SMILES'] if Chem.MolFromSmiles(smi)]

    # Generate the 2D grid image
    img_to_display = Draw.MolsToGridImage(grid_mols, molsPerRow=6, subImgSize=(300, 300))  # You can adjust `molsPerRow` as needed

    # Save and display the final image with 600 DPI
    img_filename = '/mnt/data/final_molecule_grid_600dpi.png'

    try:
        # Convert the RDKit image to a PIL image
        img_to_display_pil = PILImage.fromarray(np.array(img_to_display))  # Convert to PIL image

        # Save the image with 600 DPI resolution
        img_to_display_pil.save(img_filename, dpi=(600, 600))  # Save with 600 DPI

        # Display the saved image (optional)
        display(PILImage.open(img_filename))  # Display the saved image
        print(f"Image successfully saved to {img_filename} with 600 DPI.")
    except Exception as e:
        print(f"Error saving image: {e}")
else:
    print("No valid molecules for image generation.")


#-------------------------------------------------------------------------------------#
#                      SMILE STRUCTURE GENERATION                                     #
#-------------------------------------------------------------------------------------#


from rdkit import Chem
from rdkit.Chem import Draw
from PIL import Image as PILImage
import os
import math
import numpy as np
from IPython.display import display

# Function to generate individual 2D molecular structure images
def save_molecules_as_images(smiles_list, output_dir='/content/Final_Images', image_format='png', dpi=600):
    """
    Convert a list of SMILES strings to 2D structure images using RDKit and save them as PNG files.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Initialize a list to hold file paths for the images
    image_paths = []

    for i, smi in enumerate(smiles_list):
        mol = Chem.MolFromSmiles(smi)
        if mol is None:
            continue

        # Generate 2D coordinates (force 2D structure)
        Chem.rdDepictor.Compute2DCoords(mol)

        # Create the image for the molecule
        img = Draw.MolToImage(mol, size=(300, 300))

        # Define output file path
        output_file = os.path.join(output_dir, f"molecule_{i + 1}.{image_format}")
        img.save(output_file, dpi=(dpi, dpi))  # Save with the specified DPI
        image_paths.append(output_file)  # Add to the list

        print(f"Saved molecule {i + 1} to {output_file}")

    return image_paths

# Function to create the molecule image matrix from individual images
def create_molecule_matrix(image_paths, mols_per_row=6, tile_size=(300, 300)):
    """
    Arrange molecule images into a matrix and save as a single PNG image.
    """
    num_molecules = len(image_paths)
    num_rows = math.ceil(num_molecules / mols_per_row)

    # Create a blank white canvas for the matrix
    matrix_width = mols_per_row * tile_size[0]
    matrix_height = num_rows * tile_size[1]
    matrix_img = PILImage.new('RGB', (matrix_width, matrix_height), (255, 255, 255))

    # Paste each molecule image onto the matrix canvas
    for idx, image_path in enumerate(image_paths):
        row = idx // mols_per_row
        col = idx % mols_per_row
        img = PILImage.open(image_path)

        # Resize the image to match the tile size (in case of differing sizes)
        img = img.resize(tile_size)
        matrix_img.paste(img, (col * tile_size[0], row * tile_size[1]))

    # Save the final matrix image
    matrix_output_file = '/content/Final_Images/molecule_matrix.png'
    matrix_img.save(matrix_output_file, dpi=(600, 600))  # Save with 600 DPI
    print(f"Final molecule matrix saved to {matrix_output_file}")

    # Display the final image (optional)
    display(matrix_img)

    return matrix_output_file


# Extract SMILES list from the df_combined DataFrame
smiles_list = df_combined['SMILES'].tolist()

# Step 1: Generate individual images for each molecule
image_paths = save_molecules_as_images(smiles_list, output_dir='/content/Final_Images', image_format='png', dpi=600)

# Step 2: Create the matrix of all generated images
matrix_image_path = create_molecule_matrix(image_paths, mols_per_row=6, tile_size=(300, 300))

print(f"Matrix image created and saved at: {matrix_image_path}")